# Build a Chabot

Referene: https://python.langchain.com/docs/tutorials/chatbot/

## QuickStart

In [5]:
import os
import getpass

os.environ["LANGSMITH_TRACING"] = "true"

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")
if not os.environ.get("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("LangSmith API Key: ")

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo")

In [6]:
from langchain_core.messages import HumanMessage

messages = [
    HumanMessage(content="hi!"),
]
model.invoke(messages)

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f98f2718-d7ab-435b-bcce-66b06264cf59-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
from langchain_core.messages import AIMessage

messages = [
    HumanMessage(content="Hi! I'm Bob"),
    AIMessage(content="Hello Bob! How can I assist you today?"),
    HumanMessage(content="What's my name?"),
]
model.invoke(messages)

AIMessage(content='Your name is Bob.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 35, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b6150ba3-c7fc-49e8-9955-76e145ae3ca7-0', usage_metadata={'input_tokens': 35, 'output_tokens': 6, 'total_tokens': 41, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Message Persistence
* langgraph
  * for multi-turn application
  * in-memory checkpointer

In [8]:
# Define a new graph
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.checkpoint.memory import MemorySaver

workflow = StateGraph(state_schema=MessagesState)

# Define a function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# Add a node to the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [10]:
query = "Hi! I'm Bob"

from langchain_core.messages import HumanMessage
input_message = [HumanMessage(content=query)]
output = app.invoke({"messages": input_message}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hello Bob! It seems like there was a duplicate message. How can I assist you today?


In [11]:
query = "What's my name?"
input_message = [HumanMessage(content=query)]
output = app.invoke({"messages": input_message}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. How can I assist you today, Bob?


### New User/Thread
If we change the config to reference a different thread_id, we can see that it starts the conversation fresh.

In [12]:
config = {"configurable": {"thread_id": "abc234"}}

input_message = [HumanMessage(content=query)]
output = app.invoke({"messages": input_message}, config)
output["messages"][-1].pretty_print()


================================== Ai Message ==================================

I apologize, but I do not have any information about you to determine your name.


In [13]:
config = {"configurable": {"thread_id": "abc123"}}

input_message = [HumanMessage(content=query)]
output = app.invoke({"messages": input_message}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. How can I assist you today, Bob?


## Prompt templates


### system: Pirate

In [17]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
    MessagesPlaceholder(variable_name="messages"),
    ]
)

In [18]:
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [19]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim"

input_message = [HumanMessage(content=query)]
output = app.invoke({"messages": input_message}, config)
output["messages"][-1].pretty_print()


================================== Ai Message ==================================

Ahoy there, Jim! What be ye needin' help with today?


### system: language

In [21]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
    ]
)

In [ ]:
from typing import Sequence
from typing_extensions import Annotated, TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph import add_messages

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


